## Libraries

In [36]:
%load_ext autoreload
%autoreload 2
from conf import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%reload_ext autoreload

In [38]:
import win32com.client as win32
import os
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd

import pandas as pd
import requests
from requests_ntlm import HttpNtlmAuth

from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


### Traducciones condicion clima

In [39]:
weather_translate = {
    'Sunny' : 'Soleado',
    'Rain' : 'Lluvia',
    'Cloudy' : 'Nublado',
    'Parthly Cloudy' : 'Parcialmente Nublado',
    'Wind' : 'Viento',
    'Storm' : 'Tormenta',
    'Snow' : 'Nieve',
    'Overcast' : 'Nublado'
}

## Data Collection

In [40]:
query = 'Juarez'
api_key = config.API_KEY_WAPI


url = "http://api.weatherapi.com/v1/forecast.json?key="+api_key+"&q="+query+"&days=1&aqi=no&alerts=no"


In [41]:

auth = HttpNtlmAuth(config.PROXI_USER, config.PROXI_PWD)

response = requests.get(url, proxies=config.PROXIES, auth=auth)

if response.status_code == 200:
    data = response.json()
    print("Consulta realizada exitosamente")
else:
  print(f"Error: {response.status_code}, {response.text}")

Consulta realizada exitosamente


### Data Understanding

In [42]:
data.keys()

dict_keys(['location', 'current', 'forecast'])

In [43]:
data['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [44]:
len(data['forecast']['forecastday'][0]['hour'])

24

In [45]:
data['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] #fecha

'2025-09-16'

In [46]:
int(data['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #hora

1

In [47]:
data['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #condition

'Partly Cloudy '

In [48]:
data['forecast']['forecastday'][0]['hour'][0]['temp_c'] #temperatura


22.8

In [49]:
data['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] #llovera (1 si, 0 no      )

0

In [50]:
data['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] #Posbilidad de lluvia


0

In [51]:
data['forecast']['forecastday'][0]['day']['maxtemp_c'] #maxtemp

29.0

In [52]:
data['forecast']['forecastday'][0]['day']['daily_chance_of_rain'] #%lluvia

0

In [53]:
data['forecast']['forecastday'][0]['day']['condition']['text'] #condicion clima

'Overcast '

In [54]:
data['forecast']['forecastday'][0]['date'] #fecha

'2025-09-16'

### Data Transformation - Pronostico Hora por Hora

In [55]:
def get_forecast(data, i):
    fecha = data['forecast']['forecastday'][0]['hour'][i]['time'].split()[0] #fecha
    hora = int(data['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]) #hora
    condicion = data['forecast']['forecastday'][0]['hour'][i]['condition']['text'] #condition
    temp = data['forecast']['forecastday'][0]['hour'][i]['temp_c'] #temperatura
    rain = data['forecast']['forecastday'][0]['hour'][i]['will_it_rain'] #llovera (1 si, 0 no      )
    prob_rain = data['forecast']['forecastday'][0]['hour'][i]['chance_of_rain'] #Posbilidad de lluvia

    return fecha, hora, condicion, temp, rain, prob_rain

In [56]:
datos = []

for i in tqdm(range(len(data['forecast']['forecastday'][0]['hour'])), colour = 'green'):
    datos.append(get_forecast(data, i))

100%|██████████| 24/24 [00:00<00:00, 310689.19it/s]


In [57]:
col = ['fecha', 'hora', 'condicion', 'temp', 'rain', 'prob_rain']
df_weather = pd.DataFrame(datos, columns=col)
df_weather


,fecha,hora,condicion,temp,rain,prob_rain
0,2025-09-16,0,Partly Cloudy,22.8,0,0
1,2025-09-16,1,Partly Cloudy,22.7,0,0
2,2025-09-16,2,Cloudy,22.5,0,0
3,2025-09-16,3,Cloudy,22.4,0,0
4,2025-09-16,4,Cloudy,22.1,0,0
5,2025-09-16,5,Cloudy,22.1,0,0
6,2025-09-16,6,Cloudy,22.0,0,0
7,2025-09-16,7,Overcast,21.4,0,0
8,2025-09-16,8,Fog,21.6,0,0
9,2025-09-16,9,Partly Cloudy,22.4,0,0


In [58]:
df_rain = df_weather[(df_weather['rain'] == 1) & (df_weather['hora']> 6) & (df_weather['hora'] < 22)]
df_rain = df_rain[['hora', 'condicion']]
df_rain.set_index('hora', inplace = True)

In [59]:
df_rain

,condicion
hora,


### Data Transformation - Version Personal

In [60]:
date = data['forecast']['forecastday'][0]['date'] #fecha
maxTemp= data['forecast']['forecastday'][0]['day']['maxtemp_c'] #maxtemp
minTemp= data['forecast']['forecastday'][0]['day']['mintemp_c'] #mintemp
chanceRain = data['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] #Posbilidad de lluvia
weatherCondition = data['forecast']['forecastday'][0]['day']['condition']['text'] #condicion clima
weatherCondition = weatherCondition.strip()
weatherConditionTranslate = weather_translate[weatherCondition]

if not weatherConditionTranslate:
    weatherConditionTranslate = weatherCondition

In [61]:
mensaje = 'Hola, este es el pronostico del clima del dia de hoy: ' + date + '\nTemperadura Maxima: '+ str(maxTemp) +' °C\nTemperadura Minima: '+ str(minTemp) +' °C\nPosibilidad de Lluvia: '+ str(chanceRain) + '%' +'\nCondicion: '+ str(weatherConditionTranslate)

In [62]:
print(mensaje)

Hola, este es el pronostico del clima del dia de hoy: 2025-09-16
Temperadura Maxima: 29.0 °C
Temperadura Minima: 20.8 °C
Posibilidad de Lluvia: 0%
Condicion: Nublado


In [63]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.TO = config.EMAIL
mail.Subject = config.SUBJECT
mail.Body = mensaje

In [64]:
mail.Send()